In [6]:
pip install PyPDF2

     |████████████████████████████████| 81kB 2.7MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=e246b6e8ed6a30a45fe7c702783bc37d1d920339eba7fd44f28f2a3046e283ce
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [7]:
import PyPDF2
import nltk
nltk.download('stopwords')
import operator
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import string
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import re
import os
nltk.download('punkt')
import collections
from sklearn.metrics.pairwise import cosine_similarity


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
data ={}

In [67]:
entries_1 =[]

In [68]:
entries = os.listdir('/content/drive/My Drive/PDF files ')

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [69]:
for f in range(len(entries)):
  entries_1.append("/content/drive/My Drive/PDF files /"+entries[f])

In [70]:
doc_data = {}
x =0

In [71]:
from nltk.stem import PorterStemmer 
alpha = list(string.ascii_lowercase)
ps = PorterStemmer() 

In [72]:
for f in entries_1:
  mypdf = open(f, mode='rb')
  pdf_document = PyPDF2.PdfFileReader(mypdf)
  first_page = pdf_document.getPage(0)
  page = first_page.extractText()
  tokens = word_tokenize(page)
  tokens = [w.lower() for w in tokens]
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words = [word for word in stripped if word.isalpha()]
  words = [w for w in words if not w in alpha]
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  words = [w for w in words if len(w)>2]
  words = [ps.stem(w) for w in words]  
  ans = " ".join(words)
  doc_data[entries[x]] = ans
  x = x+1
  if x%50 == 0:
    print(x)

50
100
150
200


In [73]:
del doc_data['Natural Language Processing with TensorFlow ( PDFDrive.com )-91.pdf'] 
del doc_data['Natural Language Processing with TensorFlow ( PDFDrive.com )-51.pdf'] 
del doc_data['Natural Language Processing with TensorFlow ( PDFDrive.com )-253.pdf'] 

In [74]:
train_data =[]
for m,n in doc_data.items():
  file_string = nltk.word_tokenize(n)
  train_data.extend(file_string)
  #print(m)

In [106]:
train_data1 =[]
name_list = {}
c = 0
for m,n in doc_data.items():
  file_string = nltk.word_tokenize(n)
  train_data1.append(file_string)
  name_list[m] = c
  c = c+1
  #print(m," ",c)

Natural Language Processing with TensorFlow ( PDFDrive.com )-22.pdf   1
Natural Language Processing with TensorFlow ( PDFDrive.com )-23.pdf   2
Natural Language Processing with TensorFlow ( PDFDrive.com )-66.pdf   3
Natural Language Processing with TensorFlow ( PDFDrive.com )-64.pdf   4
Natural Language Processing with TensorFlow ( PDFDrive.com )-31.pdf   5
Natural Language Processing with TensorFlow ( PDFDrive.com )-33.pdf   6
Natural Language Processing with TensorFlow ( PDFDrive.com )-28.pdf   7
Natural Language Processing with TensorFlow ( PDFDrive.com )-36.pdf   8
Natural Language Processing with TensorFlow ( PDFDrive.com )-50.pdf   9
Natural Language Processing with TensorFlow ( PDFDrive.com )-35.pdf   10
Natural Language Processing with TensorFlow ( PDFDrive.com )-61.pdf   11
Natural Language Processing with TensorFlow ( PDFDrive.com )-42.pdf   12
Natural Language Processing with TensorFlow ( PDFDrive.com )-43.pdf   13
Natural Language Processing with TensorFlow ( PDFDrive.com )

In [26]:
pip install rank_bm25

In [76]:
vocabulary_size = 2000

def build_dataset(words):
  # Allocate a special token for rare words
  count = [['UNK', -1]]

  # Gets only the vocabulary_size most common words as the vocabulary
  # All the other words will be replaced with UNK token
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))


  # Create an ID for each word by giving the current length of the dictionary
  # And adding that item to the dictionary
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
    
  data = list()
  unk_count = 0
    
  # Traverse through all the text we have and produce a list
  # where each element corresponds to the ID of the word found at that index
  for word in words:
    # If word is in the dictionary use the word ID,
    # else use the ID of the special token "UNK"
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  
  # update the count variable with the number of UNK occurences
  count[0][1] = unk_count
    
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  # Make sure the dictionary is of size of the vocabulary
  assert len(dictionary) == vocabulary_size

  return data, count, dictionary, reverse_dictionary

def build_dataset_with_existing_dictionary(words, dictionary):
    '''
    Here we use this function to convert word strings to IDs
    with a given dictionary
    '''
    data = list()
    for word in words:
        if word in dictionary:
          index = dictionary[word]
        else:
          index = 0  # dictionary['UNK']
        data.append(index)
    return data

# Processining training data
data, count, dictionary, reverse_dictionary = build_dataset(train_data)

In [77]:
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])

Most common words (+UNK) [['UNK', 1769], ('word', 546), ('use', 379), ('lstm', 282), ('exampl', 253)]
Sample data [67, 218, 3, 16, 60, 21, 418, 1392, 7, 16]


In [78]:
wordsunique = set(train_data)
wordsunique = list(wordsunique)

In [79]:
worddic={}

In [80]:
for doc_name,doc_content in doc_data.items():
  for word_1 in wordsunique:
    if word_1 in doc_content:
      index = doc_name
   #   position = list(np.where(np.array(doc_data[index]) == word_1))
    #  idfs = tfidf(word_1,doc_content,plottest)
      try:
        worddic[word_1].append([index])
      except:
        worddic[word_1] = []
        worddic[word_1].append([index])

In [81]:
import numpy as np


You can directly use above trained word and document embedding 

In [82]:
def build_dataset_with_existing_dictionary(words, dictionary):
    '''
    Here we use this function to convert word strings to IDs
    with a given dictionary
    '''
    words1 = words.split()
    data = list()
    for word in words1:
        if word in dictionary:
          index = dictionary[word]
        else:
          index = 0  # dictionary['UNK']
        data.append(index)
    return data

In [83]:
test_data = {}
for k,v in doc_data.items():
  #  print('Building Test Dataset for ',k,' topic')
    test_data[k] = build_dataset_with_existing_dictionary(doc_data[k],dictionary)

In [84]:
import numpy as np


In [85]:
data_index = 0

def generate_batch_cbow(batch_size, window_size):
    # window_size is the amount of words we're looking at from each side of a given word
    # creates a single batch
    
    # data_index is updated by 1 everytime we read a set of data point
    global data_index

    span = 2 * window_size + 1 # [ skip_window target skip_window ]

    # two numpy arras to hold target words (batch)
    # and context words (labels)
    # Note that batch has span-1=2*window_size columns
    batch = np.ndarray(shape=(batch_size,span-1), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    # The buffer holds the data contained within the span
    buffer = collections.deque(maxlen=span)

    # Fill the buffer and update the data_index
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    # Here we do the batch reading
    # We iterate through each batch index
    # For each batch index, we iterate through span elements
    # to fill in the columns of batch array
    for i in range(batch_size):
        target = window_size  # target label at the center of the buffer
        target_to_avoid = [ window_size ] # we only need to know the words around a given word, not the word itself

        # add selected target to avoid_list for next time
        col_idx = 0
        for j in range(span):
            # ignore the target word when creating the batch
            if j==span//2:
                continue
            batch[i,col_idx] = buffer[j] 
            col_idx += 1
        labels[i, 0] = buffer[target]

        # Everytime we read a data point,
        # we need to move the span by 1
        # to create a fresh new span
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    return batch, labels

for window_size in [1,2]:
    data_index = 0
    batch, labels = generate_batch_cbow(batch_size=8, window_size=window_size)
    print('\nwith window_size = %d:' % (window_size))
    print('    batch:', [[reverse_dictionary[bii] for bii in bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])



with window_size = 1:
    batch: [['chapter', 'lstm'], ['applic', 'imag'], ['lstm', 'caption'], ['imag', 'gener'], ['caption', 'move'], ['gener', 'multimod'], ['move', 'data'], ['multimod', 'imag']]
    labels: ['applic', 'lstm', 'imag', 'caption', 'gener', 'move', 'multimod', 'data']

with window_size = 2:
    batch: [['chapter', 'applic', 'imag', 'caption'], ['applic', 'lstm', 'caption', 'gener'], ['lstm', 'imag', 'gener', 'move'], ['imag', 'caption', 'move', 'multimod'], ['caption', 'gener', 'multimod', 'data'], ['gener', 'move', 'data', 'imag'], ['move', 'multimod', 'imag', 'text'], ['multimod', 'data', 'text', 'cope']]
    labels: ['lstm', 'imag', 'caption', 'gener', 'move', 'multimod', 'data', 'imag']


In [86]:
test_data_index = 0

def generate_test_batch(data, batch_size1):
    '''
    Generate a batch of data from the test data
    This is used to compute the document embedding
    by taking the average of all the words in a document
    '''
    global test_data_index

    batch = np.ndarray(shape=(batch_size1,), dtype=np.int32)
    # Get words starting from index 0 to span
    for bi in range(batch_size1):
        batch[bi] = data[test_data_index]
        test_data_index = (test_data_index + 1) % len(data)

    return batch

test_data_index = 0
test_batch = generate_test_batch(test_data["Natural Language Processing with TensorFlow ( PDFDrive.com )-248.pdf"], 128)
print('\nwith window_size = %d:' % (window_size))
print('    labels:', [reverse_dictionary[li] for li in test_batch.reshape(128)])


with window_size = 2:
    labels: ['peephol', 'UNK', 'connect', 'allow', 'gate', 'see', 'current', 'input', 'previou', 'weight', 'lstm', 'cell', 'connect', 'shown', 'produc', 'better', 'result', 'equat', 'would', 'look', 'like', 'ihti', 'UNK', 'chtc', 'anhwx', 'fhtf', 'UNK', 'tcf', 'ohto', 'UNK', 'tho', 'UNK', 'help', 'lstm', 'perform', 'better', 'far', 'gate', 'see', 'close', 'zero', 'thu', 'gate', 'take', 'hidden', 'state', 'consider', 'calcul', 'includ', 'cell', 'state', 'directli', 'gate', 'calcul', 'equat', 'allow', 'control', 'cell', 'state', 'perform', 'well', 'even', 'situat', 'output', 'gate', 'close', 'zero', 'peephol', 'UNK', 'connect', 'allow', 'gate', 'see', 'current', 'input', 'previou', 'weight', 'lstm', 'cell', 'connect', 'shown', 'produc', 'better', 'result', 'equat', 'would', 'look', 'like', 'ihti', 'UNK', 'chtc', 'anhwx', 'fhtf', 'UNK', 'tcf', 'ohto', 'UNK', 'tho', 'UNK', 'help', 'lstm', 'perform', 'better', 'far', 'gate', 'see', 'close', 'zero', 'thu', 'gate', 'tak

In [87]:
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [88]:
batch_size = 128 # Data points in a single batch
embedding_size = 32 # Dimension of the embedding vector.
# How many words to consider left and right.
# Skip gram by design does not require to have all the context words in a given step
# However, for CBOW that's a requirement, so we limit the window size
window_size = 4

# We pick a random validation set to sample nearest neighbors
valid_size = 16 # Random set of words to evaluate similarity on.
# We sample valid datapoints randomly from a large window without always being deterministic
valid_window = 50

# When selecting valid examples, we select some of the most frequent words as well as
# some moderately rare words as well
valid_examples = np.array(random.sample(range(valid_window), valid_size))
valid_examples = np.append(valid_examples,random.sample(range(1000, 1000+valid_window), valid_size),axis=0)

num_sampled = 32 # Number of negative examples to sample.

In [89]:
tf.reset_default_graph()

# Training input data (target word IDs). Note that it has 2*window_size columns
train_dataset = tf.placeholder(tf.int32, shape=[batch_size,2*window_size])
# Training input label data (context word IDs)
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
# Validation input data, we don't need a placeholder
# as we have already defined the IDs of the words selected
# as validation data
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

In [90]:
import math

In [91]:
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0,dtype=tf.float32))

# Softmax Weights and Biases
softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                 stddev=0.5 / math.sqrt(embedding_size),dtype=tf.float32))
softmax_biases = tf.Variable(tf.random_uniform([vocabulary_size],0.0,0.01))

In [92]:
stacked_embedings = None
print('Defining %d embedding lookups representing each word in the context'%(2*window_size))
for i in range(2*window_size):
    embedding_i = tf.nn.embedding_lookup(embeddings, train_dataset[:,i])        
    x_size,y_size = embedding_i.get_shape().as_list()
    if stacked_embedings is None:
        stacked_embedings = tf.reshape(embedding_i,[x_size,y_size,1])
    else:
        stacked_embedings = tf.concat(axis=2,values=[stacked_embedings,tf.reshape(embedding_i,[x_size,y_size,1])])

assert stacked_embedings.get_shape().as_list()[2]==2*window_size
print("Stacked embedding size: %s"%stacked_embedings.get_shape().as_list())
mean_embeddings =  tf.reduce_mean(stacked_embedings,2,keepdims=False)
print("Reduced mean embedding size: %s"%mean_embeddings.get_shape().as_list())

# Compute the softmax loss, using a sample of the negative labels each time.
# inputs are embeddings of the train words
# with this loss we optimize weights, biases, embeddings
loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=mean_embeddings,
                           labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))

Defining 8 embedding lookups representing each word in the context
Stacked embedding size: [128, 32, 8]
Reduced mean embedding size: [128, 32]


In [93]:
optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)


In [94]:
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [95]:
test_labels = tf.placeholder(tf.int32, shape=[batch_size],name='test_dataset')

mean_batch_embedding = tf.reduce_mean(tf.nn.embedding_lookup(embeddings,test_labels),axis=0)

In [96]:
num_steps = 200001
cbow_loss = []

config=tf.ConfigProto(allow_soft_placement=True)
# This is an important setting and with limited GPU memory,
# not using this option might lead to the following error.
# InternalError (see above for traceback): Blas GEMM launch failed : ...
config.gpu_options.allow_growth = True

with tf.Session(config=config) as session:
    
    # Initialize the variables in the graph
    tf.global_variables_initializer().run()
    print('Initialized')
    
    average_loss = 0
    
    # Train the Word2vec model for num_step iterations
    for step in range(num_steps):
        
        # Generate a single batch of data
        batch_data, batch_labels = generate_batch_cbow(batch_size, window_size)
        
        # Populate the feed_dict and run the optimizer (minimize loss)
        # and compute the loss
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        
        # Update the average loss variable
        average_loss += l
        
        if (step+1) % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step+1, average_loss))
            cbow_loss.append(average_loss)
            average_loss = 0
        
        # Evaluating validation set word similarities
       
    
    # Computing test documents embeddings by averaging word embeddings
    
    # We take batch_size*num_test_steps words from each document
    # to compute document embeddings
    num_test_steps = 100
    
    # Store document embeddings
    # {document_id:embedding} format
    document_embeddings = {}
    print('Testing Phase (Compute document embeddings)')
  
      # For each test document compute document embeddings
    for k,v in test_data.items():
        print('\tCalculating mean embedding for document ',k,' with ', num_test_steps, ' steps.')
        test_data_index = 0
        topic_mean_batch_embeddings = np.empty((num_test_steps,embedding_size),dtype=np.float32)
        
        # keep averaging mean word embeddings obtained for each step
        for test_step in range(num_test_steps):
            test_batch_labels = generate_test_batch(test_data[k],batch_size)
            batch_mean = session.run(mean_batch_embedding,feed_dict={test_labels:test_batch_labels})
            topic_mean_batch_embeddings[test_step,:] = batch_mean
        document_embeddings[k] = np.mean(topic_mean_batch_embeddings,axis=0)
        
    word_embedding = session.run(embeddings)

Initialized
Average loss at step 2000: 3.049781
Average loss at step 4000: 2.312295
Average loss at step 6000: 1.935265
Average loss at step 8000: 1.697444
Average loss at step 10000: 1.529032
Average loss at step 12000: 1.395401
Average loss at step 14000: 1.291591
Average loss at step 16000: 1.204886
Average loss at step 18000: 1.137412
Average loss at step 20000: 1.066162
Average loss at step 22000: 1.012944
Average loss at step 24000: 0.962491
Average loss at step 26000: 0.921808
Average loss at step 28000: 0.878655
Average loss at step 30000: 0.853082
Average loss at step 32000: 0.813545
Average loss at step 34000: 0.789870
Average loss at step 36000: 0.770197
Average loss at step 38000: 0.742126
Average loss at step 40000: 0.722660
Average loss at step 42000: 0.698715
Average loss at step 44000: 0.675248
Average loss at step 46000: 0.666880
Average loss at step 48000: 0.651788
Average loss at step 50000: 0.635734
Average loss at step 52000: 0.616311
Average loss at step 54000: 0.

In [ ]:
len(set(train_data))

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(document_embeddings)

In [ ]:
df.to_csv("document_embeddings.csv")

In [97]:
def get_cosine_similarity(feature_vec_1, feature_vec_2):    
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]


In [110]:
import gensim

In [111]:
model = gensim.models.Word2Vec(sentences=train_data,
                               min_count=5,
                               window=5,
                               iter=5000)

In [112]:
def synonyms(query):
  query_new =[]

  for f in query:
    query_new.append(f)
    a = model.most_similar(f, topn=3)
    for f1 in a:
      query_new.append(f1[0])

  return query_new


In [224]:
query = "adam optimizer"
quer_list = []
def filter_query(q):
  tokens = q.split()
  tokens = [w.lower() for w in tokens]
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in tokens]
  words = [word for word in stripped if word.isalpha()]
  words = [w for w in words if not w in alpha]
  stop_words = set(stopwords.words('english'))
  words = [w for w in words if not w in stop_words]
  words = [w for w in words if len(w)>2]
  words = [ps.stem(w) for w in words] 
  return words

In [225]:
from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(train_data1)

In [226]:
query_list= filter_query(query)
#query_list= synonyms(query_list)

In [227]:
query_list

['adam', 'optim']

In [228]:
related_docs=[]

for f in query_list:
  a = worddic[f]
  for x in a:
    related_docs.append(x[0])

In [229]:
query_embedding =[]
for f in query_list:
  query_embedding.append(word_embedding[dictionary[f]])

query_embedding_mean = np.mean(np.array(query_embedding),axis=0)

In [230]:
related_docs_unique = list(set(related_docs))

In [231]:
def similar_func(related_docs_unique,query_embedding_mean,query_list):                               

  similarity={}
  doc_scores = bm25.get_scores(query_list)
  for f in range(len(related_docs_unique)):
    
    score1 = get_cosine_similarity(document_embeddings[related_docs_unique[f]],query_embedding_mean)
    score2 = doc_scores[name_list[related_docs_unique[f]]]
    similarity[related_docs_unique[f]]= score1*score2

  similarity_sort=sorted(similarity.items(), key=operator.itemgetter(1),reverse=True)
  return(similarity_sort)

In [232]:
similar_func(related_docs_unique,query_embedding_mean,query_list)

[('Natural Language Processing with TensorFlow ( PDFDrive.com )-263.pdf',
  5.868432632373222),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-289.pdf',
  3.2790836966848866),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-120.pdf',
  1.757400047641056),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-37.pdf',
  1.4633066615451669),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-139.pdf',
  1.3320619935939384),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-79.pdf',
  1.0076470386658813),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-145.pdf',
  0.7710443721327763),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-80.pdf',
  0.7037686522758102),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-141.pdf',
  0.6592153562095205),
 ('Natural Language Processing with TensorFlow ( PDFDrive.com )-151.pdf',
  0.615167681419524),
 ('Natural Language Processing with 